In [ ]:
# !pip install flask_ngrok
# !pip install flask flask-cors
# !pip install pymongo[srv]
# !pip install gdown
# !gdown https://drive.google.com/uc?id=1YydNPwhYWaK3MbHVHZCudUgODWJYqSmC

     |████████████████████████████████| 241 kB 24.2 MB/s 
Downloading...
From: https://drive.google.com/uc?id=1YydNPwhYWaK3MbHVHZCudUgODWJYqSmC
To: /content/generic-food.csv
100% 46.4k/46.4k [00:00<00:00, 6.87MB/s]


# Libraies

In [ ]:
from flask_cors import CORS, cross_origin
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import request
from flask import jsonify
import json
import pymongo
import pandas as pd
import numpy as np

client = pymongo.MongoClient("MongoDB Connection Link HERE")
mydb = client['FoodRandomizer_db']
mycol = mydb['food_list']

# Initial Data Insertion

In [ ]:
df = pd.read_csv('generic-food.csv')

df = df[{'FOOD NAME','GROUP'}]
df.rename(columns={"FOOD NAME": "name", "GROUP": "category"}, inplace=True)

df['category'] = np.random.choice(['Meal', 'Dessert', 'Drink'], df.shape[0])

df['price'] = np.random.randint(10, 500, df.shape[0])
df['price'] = df['price'].apply(str)

df['calories'] = np.random.randint(100, 1000, df.shape[0])
df['calories'] = df['calories'].apply(str)

df['delivery'] = np.random.choice(['Yes', 'No'], df.shape[0])

df['pick'] = [0] * df.shape[0]

df = df[:20]
df
dataset = df.to_dict('records')
dataset
mycol.delete_many({})
mycol.insert_many(dataset)

# Flask Server

In [ ]:
app = Flask(__name__)
CORS(app, support_credentials=True)
run_with_ngrok(app)

# /add_food?name=Pad%20Thai&category=main%20dish&price=70&calories=400&delivery=true
@app.route('/add_food') # add food to MongoDB
def add_food():
  name = request.args.get('name') # arg -> name
  category = request.args.get('category') # arg -> category
  price = request.args.get('price') # arg -> price
  calories = request.args.get('calories') # arg -> calories
  delivery = request.args.get('delivery') # arg -> delivery
  
  ret = mycol.insert_one({'name':name,
                      'category':category,
                      'price':price,
                      'calories':calories,
                      'delivery':delivery,
                      'pick':0})
  return {'data':'Insert Successfully!'}

# /delete_item?name=Noodle&category=main%20dish&price=40&calories=400&delivery=false
@app.route('/delete_item') # delete food from MongoDB
def delete_item():
  name = request.args.get('name') # arg -> name
  category = request.args.get('category') # arg -> category
  price = request.args.get('price') # arg -> price
  calories = request.args.get('calories') # arg -> calories
  delivery = request.args.get('delivery') # arg -> delivery

  query = {'name':name,
            'category':category,
            'price':price,
            'calories':calories,
            'delivery':delivery}
  mycol.delete_one(query)
  return {'data':'Delete successfully'}


# /randomize?
@app.route('/randomize') # return randomize foods
def randomize():
  # res = mycol.find({'category': "main dish"})
  category = request.args.get('category') # arg -> category
  if category == "All":
    res = mycol.find({}).sort('pick', -1)
  else:
    res = mycol.find({'category': category}).sort('pick', -1)
    
  df = pd.DataFrame([item for item in res]).sample(n=2)
  df.pop('_id')

  res = df.to_dict('records')  
  ret = dict()
  ret['data'] = res
  return jsonify(ret)

@app.route('/increase_pick') # increase pick
def increase_pick():
  name = request.args.get('name') # arg -> name
  category = request.args.get('category') # arg -> category
  price = request.args.get('price') # arg -> price
  calories = request.args.get('calories') # arg -> calories
  delivery = request.args.get('delivery') # arg -> delivery

  query = {'name':name,
            'category':category,
            'price':price,
            'calories':calories,
            'delivery':delivery}

  res = mycol.find_one(query) 
  res['pick'] = res['pick']+1
  new_values = { "$set": { "pick": res['pick'] } }
  mycol.update_one(query, new_values)
  return {'data':'Update Successfully'}

@app.route('/get_list') # return list of foods
def get_list():
  # res = mycol.find({'category': "main dish"})
  index = int(request.args.get('index')) # arg -> index
  category = request.args.get('category') # arg -> category
  if category == "All":
    res = mycol.find({}).sort('calories', -1)
  else:
    res = mycol.find({'category': category}).sort('pick', -1)

  df = pd.DataFrame([item for item in res])
  df.pop('_id')
  res = df.to_dict('records')[index:index+10]
  ret = dict()
  ret['data'] = res
  return jsonify(ret)

@app.route('/count_type') # return count of each type
def count_type():
  res = mycol.find({})
  df = pd.DataFrame([item for item in res])
  df = df.groupby('category').count()
  df = df.drop(['_id', 'price', 'calories', 'delivery'], axis=1)
  data = df.to_dict('split')
  data.pop('columns')
  data['count'] = []
  for item in data['data']:
    data['count'].append(item[0])
  data.pop('data')

  ret = dict()
  ret['data'] = data
  return jsonify(ret)

@app.route('/get_top_pick') # return get top 5 pick
def get_top_pick():
  category = request.args.get('category') # arg -> category
  if category == "All":
    res = mycol.find({}).sort('pick', -1)
  else:
    res = mycol.find({'category': category}).sort('pick', -1)

  df = pd.DataFrame([item for item in res])
  df = df[:5]
  df = df.drop(['_id', 'category', 'price', 'calories', 'delivery'], axis=1)
  df.set_index('name', inplace=True)
  data = df.to_dict('split')
  data.pop('columns')
  data['pick'] = []
  for item in data['data']:
    data['pick'].append(item[0])
  data.pop('data')

  ret = dict()
  ret['data'] = data
  return jsonify(ret)


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1e91-34-86-212-89.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [10/Dec/2021 06:43:27] "GET /get_list?index=0&category=All HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 06:43:29] "GET /get_list?index=10&category=All HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 06:43:30] "GET /get_list?index=20&category=All HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 06:43:50] "GET /count_type HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 06:43:50] "GET /get_top_pick?category=All HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 06:43:55] "GET /randomize?category=All HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 06:43:56] "GET /increase_pick?name=Burdock&category=Meal&price=421&calories=225&delivery=No HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 06:43:58] "GET /randomize?category=All HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 06:43:59] "GET /increase_pick?name=Chives&category=Drink&price=253&calories=388&delivery=No HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 06:48:27] "GET /get_list?index=0&category=All HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 06:48:40] "GET /get_list?i

Debug

In [ ]:
import pandas as pd

client = pymongo.MongoClient("[mongoDB-conn-link]")
mydb = client['FoodRandomizer_db']
mycol = mydb['food_list'] 

index=0
res = mycol.find({})
df = pd.DataFrame([item for item in res])
df.pop('_id')
res = df.to_dict('records')
res[0:3]

[{'calories': '720',
  'category': 'Herbs and Spices',
  'delivery': 'No',
  'name': 'Angelica',
  'pick': 0,
  'price': '392'},
 {'calories': '660',
  'category': 'Vegetables',
  'delivery': 'No',
  'name': 'Savoy cabbage',
  'pick': 0,
  'price': '480'},
 {'calories': '901',
  'category': 'Herbs and Spices',
  'delivery': 'No',
  'name': 'Silver linden',
  'pick': 0,
  'price': '148'}]

In [ ]:
res = mycol.find({})
df = pd.DataFrame([item for item in res])
df = df.groupby('category').count()
df = df.drop(['_id', 'price', 'calories', 'delivery'], axis=1)
data = df.to_dict('split')
data.pop('columns')
data['count'] = []
for item in data['data']:
  data['count'].append(item[0])
data.pop('data')
data

{'count': [44,
  166,
  1,
  33,
  30,
  48,
  8,
  5,
  29,
  51,
  1,
  8,
  130,
  17,
  108,
  1,
  23,
  27,
  26,
  8,
  9,
  3,
  1,
  130],
 'index': ['Animal foods',
  'Aquatic foods',
  'Baby foods',
  'Baking goods',
  'Beverages',
  'Cereals and cereal products',
  'Cocoa and cocoa products',
  'Coffee and coffee products',
  'Confectioneries',
  'Dishes',
  'Eggs',
  'Fats and oils',
  'Fruits',
  'Gourds',
  'Herbs and Spices',
  'Herbs and spices',
  'Milk and milk products',
  'Nuts',
  'Pulses',
  'Snack foods',
  'Soy',
  'Teas',
  'Unclassified',
  'Vegetables']}

In [ ]:
query = {'name':"Angelica",
            'category':"Herbs and Spices",
            'price':"430",
            'calories':"964",
            'delivery':"No"}

res = mycol.find_one(query) 
res
# res['pick'] = res['pick']+1

# new_values = { "$set": { "pick": res['pick'] } }
# mycol.update_one(query, new_values)

In [ ]:
res = mycol.find().sort('pick', -1)
df = pd.DataFrame([item for item in res])
df = df.drop(['_id', 'category', 'price', 'calories', 'delivery'], axis=1)
df.set_index('name', inplace=True)
data = df.to_dict('split')
data.pop('columns')
data['pick'] = []
for item in data['data']:
  data['pick'].append(item[0])
data.pop('data')
data

{'index': ['Lemon verbena',
  'Garlic',
  'Savoy cabbage',
  'Silver linden',
  'Garden onion',
  'Angelica',
  'Kiwi',
  'Allium (Onion)',
  'Leek',
  'Chives'],
 'pick': [4, 2, 1, 1, 1, 0, 0, 0, 0, 0]}